In [ ]:
import os
import numpy as np
import pathlib
from pyometiff import OMETIFFReader
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
xenium_path = "/mnt/sata1/Analysis_Alex/timecourse_replicates/day8_SI_r2/xenium_output"
h_and_e = np.load(os.path.join(os.path.dirname(xenium_path), "h_and_e_alignment.npy"))

In [ ]:
h_and_e = h_and_e[:32766, :32766, :]

In [ ]:
img_fpath = pathlib.Path(os.path.join(xenium_path, "morphology_mip.ome.tif"))

reader = OMETIFFReader(fpath=img_fpath)

img_array_xenium, metadata_xenium, xml_metadata = reader.read()

In [ ]:
h_and_e = h_and_e.astype(np.uint16)

In [ ]:
img_array_xenium = img_array_xenium[:32766, :32766]
img_array_xenium = 255 * (img_array_xenium / np.max(img_array_xenium))

In [ ]:
image_chunks = []
for row in tqdm(range(256, np.shape(h_and_e)[0], 256)):
    for col in range(256, np.shape(h_and_e)[1], 256):
        right_part = h_and_e[row - 256 : row, col - 256 : col, :]
        left_part = np.dstack(
            [
                img_array_xenium[row - 256 : row, col - 256 : col],
                img_array_xenium[row - 256 : row, col - 256 : col],
                img_array_xenium[row - 256 : row, col - 256 : col],
            ]
        )
        image_chunks.append(np.hstack([left_part, right_part]))

In [ ]:
image_chunks = [i for i in image_chunks if np.mean(i[:, :, 0]) > 130]

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(42)

# Shuffle the image_chunks randomly
np.random.shuffle(image_chunks)

# Calculate the split indices
total_samples = len(image_chunks)
train_size = int(0.6 * total_samples)
test_size = int(0.2 * total_samples)

# Split the image_chunks
train_image_chunks = image_chunks[:train_size]
test_image_chunks = image_chunks[train_size : train_size + test_size]
validate_image_chunks = image_chunks[train_size + test_size :]

In [ ]:
for i, arr in enumerate(train_image_chunks):
    # Convert the numpy array to an image
    img = Image.fromarray(arr.astype("uint8"))

    # Save the image to a file
    img.save(
        os.path.join(
            "/home/amonell/piloting/pytorch-CycleGAN-and-pix2pix/datasets/histology/train",
            f"image_{i + 1}.jpg",
        )
    )

for i, arr in enumerate(test_image_chunks):
    # Convert the numpy array to an image
    img = Image.fromarray(arr.astype("uint8"))

    # Save the image to a file
    img.save(
        os.path.join(
            "/home/amonell/piloting/pytorch-CycleGAN-and-pix2pix/datasets/histology/test",
            f"image_{i + 1}.jpg",
        )
    )

for i, arr in enumerate(validate_image_chunks):
    # Convert the numpy array to an image
    img = Image.fromarray(arr.astype("uint8"))

    # Save the image to a file
    img.save(
        os.path.join(
            "/home/amonell/piloting/pytorch-CycleGAN-and-pix2pix/datasets/histology/val",
            f"image_{i + 1}.jpg",
        )
    )